In [17]:


import numpy as np
import pandas as pd
import csv

import sklearn
from sklearn.decomposition import TruncatedSVD

columns = ['id','id2','task','TaskId','rating']
frame = pd.read_csv('DataRecommenderSystem/DataVisitVisitor.csv',sep=',', names=columns)
frame.head()

,id,id2,task,TaskId,rating
0,1,3,1000,1,6
1,2,3,1110,2,6
2,3,3,1100,3,7
3,4,3,1010,4,8
4,5,3,1000000,5,7


In [18]:
columns2 =['TaskId','TaskName','Task']
tasks = pd.read_csv('DataRecommenderSystem/TaskVisitVisitor.csv', sep=',', names = columns2, encoding='latin-1')
task_names = tasks[['TaskId','TaskName','Task']]
task_names.head()


,TaskId,TaskName,Task
0,1,Task1,1000
1,2,Task2,1110
2,3,Task3,1100
3,4,Task4,1010
4,5,Task5,1000000


In [19]:
combined_task_data = pd.merge(frame, task_names, on='TaskId')
combined_task_data.head()

,id,id2,task,TaskId,rating,TaskName,Task
0,1,3,1000,1,6,Task1,1000
1,16,3,1000,1,8,Task1,1000
2,18,3,1000,1,4,Task1,1000
3,26,3,1000,1,5,Task1,1000
4,33,3,1000,1,0,Task1,1000


In [20]:
combined_task_data.groupby('TaskId')['rating'].count().sort_values(ascending=True).head()

filter_interview = combined_task_data['TaskId']==0
combined_task_data[filter_interview]['TaskName'].unique()

array([], dtype=object)

In [21]:
rating_crosstab = combined_task_data.pivot_table(values='rating', index='id', columns=['TaskName','Task'], fill_value=0.01)
rating_crosstab

TaskName,Task1,Task10,Task11,Task2,Task3,Task4,Task5,Task6,Task7,Task8,Task9
Task,1000,1000010,1000110,1110,1100,1010,1000000,1101010,101010,100010,100110
id,,,,,,,,,,,
1,6.00,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01
2,0.01,0.01,0.01,6.00,0.01,0.01,0.01,0.01,0.01,0.01,0.01
3,0.01,0.01,0.01,0.01,7.00,0.01,0.01,0.01,0.01,0.01,0.01
4,0.01,0.01,0.01,0.01,0.01,8.00,0.01,0.01,0.01,0.01,0.01
5,0.01,0.01,0.01,0.01,0.01,0.01,7.00,0.01,0.01,0.01,0.01
...,...,...,...,...,...,...,...,...,...,...,...
96,0.01,0.01,0.01,0.01,4.00,0.01,0.01,0.01,0.01,0.01,0.01
97,0.01,0.01,0.01,5.00,0.01,0.01,0.01,0.01,0.01,0.01,0.01


In [22]:
rating_crosstab.shape
X = rating_crosstab.T
X.shape

(11, 100)

In [23]:
SVD = TruncatedSVD(n_components=11, random_state=0)

resultant_matrix = SVD.fit_transform(X)

resultant_matrix.shape

(11, 11)

In [24]:
corr_mat = np.corrcoef(resultant_matrix)
corr_mat.shape
task_names = rating_crosstab.columns
task_list = list(task_names)
task_list
task = task_list.index(('Task1',1000))
task
corr_1 = corr_mat[task]
corr_1.shape
corr_1

with open('DataRecommenderSystem/recommenderSystemVisit.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(corr_1)
    writer.writerow(list(task_names[(corr_1<=1) & (corr_1 > -0.09)]))


